In [ ]:
# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

### Metrics to consider:
- Monthly active users
- Daily active users in past month
- Total paid and unpaid users
- Total ads served in the past month
- Cohort per Month - % of users cancelled, % of users upgrades


In [ ]:
# Import Libraries

In [309]:
# import libraries
import seaborn as sns
import pandas as pd
import numpy as np
import datetime # missing and together with "pip install --upgrade pyspark" were causing issues with time series
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from plotly.offline import iplot
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import cufflinks as cf
cf.go_offline()
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [ ]:
# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [ ]:
# create a Spark session
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
print(spark.catalog.listTables())

In [ ]:
pwd

In [ ]:
mini = '/home/freemo/Projects/largeData/mini_sparkify_event_data.json'
medium = '/home/freemo/Projects/largeData/medium_sparkify_event_data.json'

In [ ]:
df = spark.read.json(mini)

In [ ]:
df.take(1)

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
# check columns with Null values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
# missing values in userID
df.select([count(when(isnan('userID'),True))]).show()

In [ ]:
# missing values in sessionID
df.select([count(when(isnan('sessionID'),True))]).show()

In [ ]:
# get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)
# df = df.withColumn("hour", get_hour(df.ts))
# songs_in_hour = df.filter(df.page == "NextSong").groupBy(df.hour).count().orderBy(df.hour.cast("float"))
# songs_in_hour.show()

# songs_in_hour_pd = songs_in_hour.toPandas()
# songs_in_hour_pd.hour = pd.to_numeric(songs_in_hour_pd.hour)

In [ ]:
# create udf (user defined functions) for workdays and hour columns
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
day_ind = [1,2,3,4,5,6,7]
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).hour, IntegerType() )
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).day, IntegerType() )
get_wkday = udf(lambda x: day_name[datetime.datetime.fromtimestamp(x/1000).weekday()] )
get_month = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).month, IntegerType() )

In [ ]:
# create new columns based on ts column for analyses
df = df.withColumn('hour', get_hour('ts'))
df = df.withColumn('day', get_day('ts'))
df = df.withColumn('workday', get_wkday('ts'))
df = df.withColumn('month', get_month('ts'))

In [ ]:
df.select('hour', 'day', 'workday', 'month').show(7)

In [ ]:
# df.groupBy('ts_hour').count().show()

In [ ]:
# df.groupBy('ts_day').count().show(31)

In [ ]:
user_log_valid = df.dropna(how = "any", subset = ["userId", "sessionId"])

In [ ]:
user_log_valid.count()

In [ ]:
df.select("userId").dropDuplicates().sort("userId").show()

In [ ]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [ ]:
user_log_valid.count() - 286500

In [ ]:
# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

In [ ]:
### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

In [ ]:
# check the page column
df.select("page").dropDuplicates().sort("page").show()

In [ ]:
# How many songs do users listen to on average between visiting our home page? 

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

In [ ]:
# top 3 played artist
df.filter(df.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(3)

In [ ]:
# check a random user
df.select(["userId", "firstname", "page", "level", "song"]).where(df.userId == 98).collect()

In [ ]:
# create a udf to flag Churned customers
flag_downgrade_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [ ]:
# create new column "churned"
df = df.withColumn("churn", flag_downgrade_event("page"))

In [ ]:
df.head()

In [ ]:
# add a staging: stage=1 if the user is paid, stage=2 if the user churned
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

In [ ]:
# create new column stage to assign values
df = df.withColumn("phase", Fsum("churn").over(windowval))

In [ ]:
# check the number of songs played per user
df.where(df.song != 'null').groupBy(['churn','userId']) \
    .agg(count(df.song).alias('SongsPlayed')).sort('SongsPlayed', ascending=False).show()

In [ ]:
User 39 played the biggest number of songs. Let's check if that matters by going through it's records.

In [ ]:
# check result
df.select(["userId", "firstname", "ts", "page", "level", "phase"]).where(df.userId == "39").sort("ts").collect()

In [ ]:
User 39 started as free, but after some decent amount of songs listened it became a paid member. 

In [ ]:
### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
df.dropDuplicates(["userId"]).groupBy(['gender','churn']).count().show()

In [ ]:
df.dropDuplicates(['userId']).groupBy(['song','churn']).count().show()

In [ ]:
df.where(df.song != 'null').groupBy(['churn','userId']) \
    .agg(count(df.song).alias('SongsPlayed')).sort('SongsPlayed', ascending=False).show()

In [ ]:
df.printSchema()

In [ ]:
# plot locations 
# On which days do people use sparkify more?
# What are the most active hours?
# Songs played per user group - churned/not churned
# Which days of the month are more active?
# Which subscription level is more likely to churn?
# Do activity times have effect over churn?
# https://hendra-herviawan.github.io/pyspark-groupBy-and-aggregate-functions.html
# https://github.com/ihsankose/Udacity-Capstone/blob/master/Sparkify.ipynb
# https://nbviewer.jupyter.org/github/elifinspace/sparkify/blob/master/Sparkify_final_.ipynb

In [ ]:
1. Try to aggregate based on songs and month
2. Plot some EDA based on notes
3. Clean data set for modeling (refer to previous projects with ETL pipelines)
4. Model based on DataCamp course (if possible|)

In [ ]:
plot = df.groupBy('location', "churn").count().dropDuplicates().sort('count', ascending=False).toPandas()

In [ ]:
fig = px.bar(plot, x='location', y='count')
fig.show()

In [ ]:
# create state column
df = df.withColumn("state", df.location.substr(-2,2))

In [ ]:
df.select('state').show()

In [ ]:
df.groupBy("state").count().sort("count", ascending=False).show(5)

In [ ]:
# convert to pandas
state = df.groupBy("state", "churn").count().sort("count", ascending=False).toPandas()

In [ ]:
fig = px.bar(state, x='state', y='count')
fig.show()

In [ ]:
churn = df.filter(df.churn == 1)
churn.groupBy("state", "churn").count().sort("count", ascending=False).show(5)

In [ ]:
churned_state = churn.groupBy("state", "churn").count().sort("count", ascending=False).toPandas()

In [ ]:
fig = px.bar(churned_state, x='state', y='count')
fig.show()

In [ ]:
# churn
temp = churn.groupBy('workday').count().sort('count').toPandas()
fig = px.bar(temp, x='workday', y='count', text="count")
fig.update_layout(title_text='What are the most active days for churn?')
fig.show()

In [ ]:
churn.groupBy('song', "churn").count().dropDuplicates().sort('count', ascending=False).show()

In [ ]:
**No songs for churned users**

In [ ]:
# what are the most active days?
temp = df.groupBy('workday').count().sort('count').toPandas()
fig = px.bar(temp, x='workday', y='count', text="count")
fig.update_layout(title_text='What are the most active days?')
fig.show()

In [ ]:
# What are the most active hours?
temp = df.groupBy('hour').count().sort('hour').toPandas()
fig = px.bar(temp, x='hour', y='count', text='count')
fig.update_layout(title_text='What are the most active hours?')
fig.show()

In [ ]:
# What are the most active hours for churn?
temp = churn.groupBy('hour').count().sort('hour').toPandas()
fig = px.bar(temp, x='hour', y='count', text='count')
fig.update_layout(title_text='What are the most active hours for churn?')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

In [ ]:
# What are the most active days during the month?
temp = df.groupBy('day').count().sort('day').toPandas()
fig = px.bar(temp, x='day', y='count', text='count')
fig.update_layout(title_text='What are the most active days during the month?')
fig.show()

In [ ]:
# What are the most active days during the month for churn?
temp = churn.groupBy('day').count().sort('day').toPandas()
fig = px.bar(temp, x='day', y='count', text='count')
fig.update_layout(title_text='What are the most active days during the month?')
fig.show()

In [ ]:
# How people are split per gender?
temp = df.groupBy('gender').count().sort('gender').toPandas()
fig = px.bar(temp, x='gender', y='count', text='count')
fig.update_layout(title_text='Gender split')
fig.show()

In [ ]:
# How people are split per gender?
temp = churn.groupBy('gender').count().sort('gender').toPandas()
fig = px.bar(temp, x='gender', y='count', text='count')
fig.update_layout(title_text='Gender split')
fig.show()

In [ ]:
Eventhough more users are female, it turns out that Men are churning the most.

In [ ]:
### Based on the EDA I would choose the below features:
- state
- workday
- day
- songs played
- gender
- level

In [ ]:
# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
df.printSchema()

In [ ]:
df.select('gender', 'userId', 'workday', 'day', 'churn', 'state').show()

In [ ]:
# create state column
# df = df.withColumn("state", df.location.substr(-2,2))

# create udf (user defined functions) for workdays and hour columns
#day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
#day_ind = [1,2,3,4,5,6,7]
#get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).hour, IntegerType() )
#get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).day, IntegerType() )
#get_wkday = udf(lambda x: day_name[datetime.datetime.fromtimestamp(x/1000).weekday()] )
#get_month = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).month, IntegerType() )

# create new columns based on ts column for analyses
#df = df.withColumn('hour', get_hour('ts'))
#df = df.withColumn('day', get_day('ts'))
#df = df.withColumn('workday', get_wkday('ts'))
#df = df.withColumn('month', get_month('ts'))

# check the number of songs played per user
#df.where(df.song != 'null').groupBy(['churn','userId']) \
    #.agg(count(df.song).alias('SongsPlayed')).sort('SongsPlayed', ascending=False).show()

In [ ]:
### Feature 1 - gender

In [ ]:
gender = df.dropDuplicates(['userId']).sort('userId').select(['userId','gender'])
gender = gender.filter(gender["userId"] != "")
gender = gender.replace(['M','F'], ['1', '0'], 'gender')
gender = gender.withColumn('gender', gender.gender.cast("int"))

In [ ]:
gender.take(1)

In [ ]:
### Feature 2 - workday

In [ ]:
workday = df.dropDuplicates(['userId']).sort('userId').select(['userId','workday'])
workday = workday.filter(workday["userId"] != "")

In [ ]:
workday.take(1)

In [ ]:
### Feature 3 - day

In [ ]:
day = df.dropDuplicates(['userId']).sort('userId').select(['userId','day'])
day = day.filter(day["userId"] != "")

In [ ]:
day.take(1)

In [ ]:
### Feature 4 - state

In [ ]:
state = df.dropDuplicates(['userId']).sort('userId').select(['userId','state'])
state = state.filter(state["userId"] != "")

In [ ]:
state.take(1)

In [ ]:
### Feature 5 - SongsPlayed

In [ ]:
songs_played = df.where(df.song!='null').groupby('userId') \
    .agg(count(df.song).alias('SongsPlayed')).orderBy('userId').select(['userId','SongsPlayed'])
songs_played = songs_played.filter(songs_played["userId"] != "")

In [ ]:
songs_played.take(1)

In [ ]:
### Features data frame

In [287]:
# create features data frame
model_data = df.dropDuplicates(['userId']).select(['userId','churn'])
model_data = model_data.filter(state["userId"] != "")

In [288]:
model_data.take(1)

[Row(userId='100010', churn=0)]

In [289]:
# join all features into one data frame
model_data = model_data.join(gender, 'userId')
model_data = model_data.join(workday, 'userId')
model_data = model_data.join(day, 'userId')
model_data = model_data.join(state, 'userId')
model_data = model_data.join(songs_played, 'userId')

In [290]:
model_data.show()

+------+-----+------+---------+---+-----+-----------+
|userId|churn|gender|  workday|day|state|SongsPlayed|
+------+-----+------+---------+---+-----+-----------+
|100010|    0|     0|   Monday|  8|   CT|        275|
|200002|    0|     1|   Monday|  1|   WI|        387|
|   125|    0|     1|   Friday| 12|   TX|          8|
|   124|    0|     0|   Monday|  1|   NJ|       4079|
|    51|    0|     1|   Monday|  1|   KY|       2111|
|     7|    0|     1|Wednesday|  3|   WA|        150|
|    15|    0|     1|   Monday|  1|   WV|       1914|
|    54|    0|     0|   Monday|  1|   WA|       2841|
|   155|    0|     0|   Friday|  2|   NH|        820|
|100014|    0|     1|Wednesday| 10|   PA|        257|
|   132|    0|     0|  Tuesday|  2|   CO|       1928|
|   154|    0|     0|   Friday|  2|   PA|         84|
|   101|    0|     1|   Monday|  1|   CO|       1797|
|    11|    0|     0|   Friday|  5|   MA|        647|
|   138|    0|     1| Saturday|  6|   PA|       2070|
|300017|    0|     0|   Mond

In [312]:
model_data = model_data.withColumn("label", model_data.churn)

In [313]:
model_data.show()

+------+-----+------+---------+---+-----+-----------+-----+
|userId|churn|gender|  workday|day|state|SongsPlayed|label|
+------+-----+------+---------+---+-----+-----------+-----+
|100010|    0|     0|   Monday|  8|   CT|        275|    0|
|200002|    0|     1|   Monday|  1|   WI|        387|    0|
|   125|    0|     1|   Friday| 12|   TX|          8|    0|
|   124|    0|     0|   Monday|  1|   NJ|       4079|    0|
|    51|    0|     1|   Monday|  1|   KY|       2111|    0|
|     7|    0|     1|Wednesday|  3|   WA|        150|    0|
|    15|    0|     1|   Monday|  1|   WV|       1914|    0|
|    54|    0|     0|   Monday|  1|   WA|       2841|    0|
|   155|    0|     0|   Friday|  2|   NH|        820|    0|
|100014|    0|     1|Wednesday| 10|   PA|        257|    0|
|   132|    0|     0|  Tuesday|  2|   CO|       1928|    0|
|   154|    0|     0|   Friday|  2|   PA|         84|    0|
|   101|    0|     1|   Monday|  1|   CO|       1797|    0|
|    11|    0|     0|   Friday|  5|   MA

In [314]:
# check column types
model_data.dtypes

[('userId', 'int'),
 ('churn', 'int'),
 ('gender', 'int'),
 ('workday', 'string'),
 ('day', 'int'),
 ('state', 'string'),
 ('SongsPlayed', 'bigint'),
 ('label', 'int')]

In [316]:
model_data = model_data.withColumn("userId", model_data.userId.cast("integer"))

In [317]:
# Create a StringIndexer
state_indexer = StringIndexer(inputCol ="state", outputCol ="state_index")

In [318]:
# Create a OneHotEncoder
state_encoder = OneHotEncoder(inputCol="state_index", outputCol="state_fact")

In [319]:
# Create a StringIndexer
workday_indexer = StringIndexer(inputCol ="workday", outputCol ="workday_index")

In [320]:
# Create a OneHotEncoder
workday_encoder = OneHotEncoder(inputCol="workday_index", outputCol="workday_fact")

In [321]:
# Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["userId", "gender", "state_fact", "workday_fact", "day"], outputCol="features")

In [ ]:
# Remove missing values
model_data = model_data.filter("arr_delay is not NULL \
and dep_delay is not NULL \
and air_time is not NULL \
and plane_year is not NULL")

In [ ]:
ETL pipeline

1. Load data
2. Create workdays/days columns
3. Create features
4. Join data

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [322]:
from pyspark.ml import Pipeline

# Make the pipeline
churn_pipe = Pipeline(stages=[state_indexer, state_encoder, workday_indexer, workday_encoder, vec_assembler])

In [323]:
piped_data = churn_pipe.fit(model_data).transform(model_data)

In [324]:
# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

In [325]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

In [326]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [327]:
# Import the tuning submodule
import pyspark.ml.tuning as tune


# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()


In [328]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
            estimatorParamMaps=grid,
            evaluator=evaluator
            )

In [329]:
# Fit cross validation models
models = cv.fit(training)

# Extract the best model
best_lr = models.bestModel

IllegalArgumentException: requirement failed: rawPredictionCol vectors must have length=2, but got 1

In [ ]:
# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.